<a href="https://colab.research.google.com/github/juanestebanperezm/tf_imagenes/blob/main/tf_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import io
import os
import re
import shutil
import string

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, TextVectorization



In [7]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

datos = keras.utils.get_file(
                            "aclImdb_v1.tar.gz",
                            url, 
                            untar = True, 
                            cache_dir = ".", 
                            cache_subdir = ""
                             )

datos_dir = os.path.join(os.path.dirname(datos),'aclImdb')

os.listdir(datos_dir)

['imdb.vocab', 'test', 'README', 'train', 'imdbEr.txt']

In [9]:
entrenamiento_dir = os.path.join(datos_dir,'train')
remover_dir = os.path.join(entrenamiento_dir,'unsup')
shutil.rmtree(remover_dir)

In [11]:
batch_size = 1024
seed = 123

entrenamiento_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = "training",
    seed = seed
)

validacion_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = "validation",
    seed = seed

)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

entrenamiento_ds = entrenamiento_ds.cache().prefetch( buffer_size = AUTOTUNE )
validacion_ds = validacion_ds.prefetch( buffer_size = AUTOTUNE  )

In [13]:
capa_embebida = Embedding(1000,5)

In [22]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />", " " )
  return tf.strings.regex_replace( 
        stripped_html, 
        "[%s]"%re.escape(string.punctuation),
        "" 
        )
  

total_vocabulario = 10000
longitud_secuencia = 100

capa_vectorizada = TextVectorization(
    standardize = custom_standardization,
    max_tokens = total_vocabulario,
    output_mode = "int",
    output_sequence_length = longitud_secuencia
)

texto_ds = entrenamiento_ds.map( lambda x , y : x )
capa_vectorizada.adapt(texto_ds)

In [25]:
dimensiones_embebidas = 16

modelo = Sequential( 
                    [
                      capa_vectorizada,
                      Embedding(total_vocabulario, dimensiones_embebidas ,name = "embebida"),
                      GlobalAveragePooling1D(),
                      Dense(16,activation = "relu"),
                      Dense(1)
                    ] 
                    )

In [27]:
tensor_llamada = keras.callbacks.TensorBoard(log_dir = "logs")


modelo.compile(
    optimizer = "adam",
    loss = keras.losses.BinaryCrossentropy( from_logits = True ),
    metrics = ["accuracy"]
)

In [28]:
modelo.fit(
    entrenamiento_ds,
    validation_data = validacion_ds,
    epochs = 15,
    callbacks = [tensor_llamada]
)

Epoch 1/15
20/20 [==============================] - 6s 110ms/step - loss: 0.6918 - accuracy: 0.5028 - val_loss: 0.6896 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 99ms/step - loss: 0.6870 - accuracy: 0.5028 - val_loss: 0.6830 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 108ms/step - loss: 0.6784 - accuracy: 0.5028 - val_loss: 0.6722 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 113ms/step - loss: 0.6648 - accuracy: 0.5028 - val_loss: 0.6564 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 107ms/step - loss: 0.6455 - accuracy: 0.5087 - val_loss: 0.6356 - val_accuracy: 0.5124
Epoch 6/15
20/20 [==============================] - 2s 105ms/step - loss: 0.6208 - accuracy: 0.5643 - val_loss: 0.6108 - val_accuracy: 0.5898
Epoch 7/15
20/20 [==============================] - 2s 106ms/step - loss: 0.5916 - accuracy: 0.6471 - val_loss: 0.5833 - val_accuracy: 0.6560
Epoch 8